# Example: collocation of a simple growth model with Markov productivity shock

In [ ]:
import AdaptiveSG as asg # check my package `AdaptiveSG.jl` on github

col = include("src/Collocations.jl") # in case you did not install the package

println(names(col))

[:Collocations, :MarkovCollocationUniform, :precond!, :update_θ!]


## Model

$$
\begin{aligned}
& v(k,z) = \max_{c,k'} \log{c} + \beta \mathbb{E}\{ v(k',z') | z\} \\
& k' = z \cdot k^\alpha - \delta k - c \\
& c > 0, k' > 0 \\
& \log{z} \sim \text{MarkovChain}
\end{aligned}
$$

Parameterization:

- $\alpha = 0.3$
- $\delta = 0.05$
- $\beta = 0.9$

$$
z \in \{0.8, 1.2\}; P = \begin{bmatrix}
0.7 & 0.3 \\
0.5 & 0.5
\end{bmatrix}
$$


Assumes the current policy guess $c(k,z) = (1-\beta) (zk^\alpha - \delta k)$.
Iterate the value function only.

In [ ]:
# step: initialize the value function interpolation as an RSG
# NOTE: we only need the grid structure and train an initial guess for the value function
#       these are shared across all 2 states of `z`
vitp, nzer, pars = let accuracy = 8

    local _pars = (
        α = 0.3,
        δ = 0.05,
        β = 0.9,
        z = [0.8, 1.2],
        Pz = [0.7 0.3; 0.5 0.5],
    )

    # NOTE: only the endo state(s) (k,)!
    local _nzer = asg.Normalizer{1}((0.01,), (2.0,))

    local _vitp = asg.RegularSparseGrid{1}(
        accuracy, 
        ntuple(_ -> accuracy, 1),
    )

    # guess: the initial value function to iterate on
    asg.train!(
        _vitp,
        X01 -> begin
            k = asg.denormalize(X01, _nzer)[1]
            c = (1.0 * k^_pars.α - _pars.δ * k) * (1.0 - _pars.β)
            return log(c) / (1.0 - _pars.β)
        end,
        printlevel = "final",
        validate_io = false
    )
    
    _vitp, _nzer, _pars
end # let

The RSG is trained.


(RegularSparseGrid{1}(depth = 8, #nodes = 129, max_levels = (8,)), Normalizer{1}
	x[1] in [0.01, 10.0]
, (α = 0.3, δ = 0.05, β = 0.9, z = [0.8, 1.2], Pz = [0.7 0.3; 0.5 0.5]))

In [110]:
# define the collocation model
mcl = col.MarkovCollocationUniform(
    asg.basis_matrix(vitp),
    pars.Pz,
    D = 1, J = 1,
    dimnames = (:k,),
    β = pars.β,
)
display(mcl)

UniformMarkovCollocation{D=1,N=129,K=2,J=1}
- dimnames of x states        : [:k]
- dimension of endog states   : D = 1
- dimension of exog shocks    : J = 1
- discount factor             : β = 0.9
- # of grid nodes             : N = 129
- # of states in Markov chain : K = 2
- # of v(x,z)       interp coefficients N*K = 258
- # of E{v(x,z')|z} interp coefficients N*K = 258
- size of Jacobian matrix : (516, 516)


In [111]:
# load the initial guess of θs and θEs
# STRATEGY: use the same initial guess for both θs and θEs, for all `z` states
mcl.θs  = repeat(vitp |> asg.interpcoef, outer = (1, 2)) # 2 states of `z`
mcl.θEs = repeat(vitp |> asg.interpcoef, outer = (1, 2)) # 2 states of `z`


# pre-conditioning the collocation model
col.precond!(mcl)


# validate the model after pre-conditioning
col.validate!(mcl)


display(mcl.stackΦXperm)

258×258 SparseArrays.SparseMatrixCSC{Float64, Int64} with 1798 stored entries:
⎡⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⣿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣧⠳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡏⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡿⡄⢣⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⢸⠈⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡇⢣⠀⠳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡇⣇⠈⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⠘⡄⠀⢣⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⢇⢸⠀⠈⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⠀⢧⠀⠀⠳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⢸⠈⡇⠀⠈⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡇⡇⠸⡄⠀⠀⢣⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡼⡄⢸⠀⠀⠈⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡇⢇⠀⢣⠀⠀⠀⠳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡇⡇⠀⣇⠀⠀⠈⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡇⢸⠀⠘⡄⠀⠀⠀⢣⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡇⢣⠀⢸⠀⠀⠀⠈⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⠸⡄⠀⢧⠀⠀⠀⠀⠳⡀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡇⢸⠀⠈⡇⠀⠀⠀⠈⢣⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⠀⡇⠀⠘⡄⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⣿⢳⢸⠀⠀⢸⠀⠀⠀⠀⠈⢦⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⠀⡇⠀⠀⢣⠀⠀⠀⠀⠀⠳⡀⠀⠀⠀⠀⠀⠀⣿⢸⠀⡇⠀⠀⣇⠀⠀⠀⠀⠈⢣⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡿⡀⢹⠀⠀⠘⡆⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⣿⢸⠀⢇⠀⠀⢸⠀⠀⠀⠀⠀⠈⢦⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡇⡇⢸⠀⠀⠀⢧⠀⠀⠀⠀⠀⠀⠳⡀⠀⠀⠀⠀⣿⢸⠀⢸⠀⠀⠈⡇⠀⠀⠀⠀⠀⠈⢣⠀⠀⠀⠀⠀⎥
⎢⣿⡇⡇⠀⡇⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⣿⡇⡇⠸⡄⠀⠀⢸⠀⠀⠀⠀⠀⠀⠈⢦⠀⠀⠀⠀⎥
⎢⣿⡇⡇⠀⢇⠀⠀⠀⢣⠀⠀⠀⠀⠀⠀⠀⠳⡀⠀⠀⣿⡇⡇⠀⡇⠀⠀⠀⣇⠀⠀⠀⠀⠀⠀⠈⢣⠀⠀⠀⎥
⎢⣿⡇⢳⠀⢸⠀⠀⠀⠘⡆⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⣿⡇⡇⠀⢣⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠈⢦⠀⠀⎥
⎢⣿⡇⢸⠀⠸⡄⠀⠀⠀⢧⠀⠀⠀⠀⠀⠀⠀⠀⠳⡀⣿⡇⢇⠀⢸⠀⠀⠀⠈⡇⠀⠀⠀⠀⠀⠀⠀⠈⢣⠀⎥
⎣⣿⡇⢸⠀⠀⡇⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⢳⣿⡇⢸⠀⢸⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠈⢦⎦

In [ ]:
# Run: fixed point iteration
errTrace, Xnodes, Xnext = let maxiter = 200, showevery = 20, atol = 1E-4

    # shape parameters
    local N = size(mcl.ΦX, 1)
    local K = size(mcl.Pz, 1)
    local D = 1 # (k,)
    local J = 1 # (z,)

    # malloc: the old guess of (θs, θEs); for computing the convergence criterion
    local θs_old = copy(mcl.θs)
    local θEs_old = copy(mcl.θEs)

    # malloc: the optimal U(x,c) and Xnext(x,c) for all `z` states, and Φ(Xnext)
    local Us  = similar(mcl.Us)
    local Xp  = [
        zeros(N, D)
        for _ in 1:K
    ]
    local ΦXp = [
        col.spzeros(size(mcl.ΦX))
        for _ in 1:K
    ]

    # pre-cond: the materialized X grid nodes
    local Xnodes = [
        asg.denormalize(xi, nzer)
        for xi in asg.vectorize_x(vitp) |> eachrow
    ] |> stack |> permutedims
    

    # malloc: error trace
    local errTrace = []


    @time for t in 1:maxiter

        # step: useful intermediate variables
        inc = pars.z' .* Xnodes[:,1] .^ pars.α .- pars.δ .* Xnodes[:,1]

        # step: optimization step (in this example, take it as given)
        cOpt = zeros(N,K)
        for k in 1:K
            cOpt[:,k] .= (1 - pars.β) .* inc[:,k]
        end

        # eval: U(X) stackings
        for k in 1:K
            Us[:,k] .= log.(cOpt[:,k] .+ 1E-6)
        end

        # eval: x' = 𝔛(x,z;c), the state equation; be careful about the state constraints
        for k in 1:K
            Xp[k] .= pars.z[k] .* Xnodes[:,1] .^ pars.α .- pars.δ .* Xnodes[:,1] .- cOpt[:,k]
            clamp!(
                Xp[k],
                nzer.lb[1],
                nzer.ub[1]
            )
        end

        # eval: Φ(X'), be careful about the scaling due to ASG
        for k in 1:K
            for i in 1:N
                ΦXp[k][i,:] = asg.basis_matrix(
                    asg.normalize(
                        Xp[k][i,:],
                        nzer
                    ),
                    vitp
                )
            end
        end

        # load: the stacking U(X), and {Φ(X'|z)} to the collocation model
        mcl.Us = Us
        mcl.ΦXnext = ΦXp

        # solve: the fixed point problem
        col.update_θ!(mcl)

        # compute the convergence criterion
        err4z_θs = maximum(
            abs,
            mcl.θs .- θs_old, 
            dims = 1
        ) |> vec
        err4z_θEs = maximum(
            abs,
            mcl.θEs .- θEs_old, 
            dims = 1
        ) |> vec

        errAgg = max(
            maximum(err4z_θs),
            maximum(err4z_θEs)
        )
        push!(errTrace, errAgg)

        # report
        if t % showevery == 0
            println("t = $t")
            println("\t- errAgg = $errAgg")
            println("\t- err4z_θs = $err4z_θs")
            println("\t- err4z_θEs = $err4z_θEs")
        end

        # check convergence
        if errAgg < atol
            println("Converged in iteration $t !")
            break
        end

        # update the old guess
        copyto!(θs_old, mcl.θs)
        copyto!(θEs_old, mcl.θEs)

    end # t

    errTrace, Xnodes, Xp
end; # let

t = 20
	- errAgg = 0.031350645122593335
	- err4z_θs = [8.881784197001252e-15, 8.881784197001252e-15]
	- err4z_θEs = [0.031350645122593335, 0.025851096675289398]
t = 40
	- errAgg = 0.003811506552651167
	- err4z_θs = [7.105427357601002e-15, 8.881784197001252e-15]
	- err4z_θEs = [0.003811506552651167, 0.0031428898517340542]
t = 60
	- errAgg = 0.0004633902156214731
	- err4z_θs = [8.881784197001252e-15, 1.4210854715202004e-14]
	- err4z_θEs = [0.0004633902156214731, 0.00038210203390676156]
Converged in iteration 75 !
  0.084298 seconds (905.57 k allocations: 420.763 MiB, 15.42% gc time, 26.00% compilation time)


In [113]:
# visualize the convergence
import UnicodePlots as up

up.lineplot(
    log.(errTrace), 
    title = "log(aggregate error)", 
    xlabel = "t", ylabel = "",
    xlim = (1, length(errTrace))
)

       ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀log(aggregate error)⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
       ┌────────────────────────────────────────┐ 
    10 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⢣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠼⡤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤│ 
       │⠀⠣⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠉⠑⠒⠤⠤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠓⠒⠤⠤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠒⠢⠤⢄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠒⠢⠤⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠑⠒⠒⠤⢄⣀⠀⠀⠀│ 
   -10 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠑⠒│ 
       └────────────────────────────────────────┘ 
       ⠀1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀75⠀ 
       ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀t⠀⠀

In [114]:
# visualize the value function
up.scatterplot(Xnodes[:], mcl.ΦX * mcl.θs[:,1], title = "Value function", xlabel = "k")

       ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Value function⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
       ┌────────────────────────────────────────┐ 
   -10 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⣀⣀⣀⡤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤│ 
       │⠀⠀⠀⠀⠀⠀⠀⢀⣀⠤⠤⠔⠒⠒⠋⠉⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⣀⠤⠒⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⢀⠜⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠠⠊⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
   -40 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       └────────────────────────────────────────┘ 
       ⠀0⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀10⠀ 
       ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀k⠀⠀

In [115]:
# visualize the expected value function
up.scatterplot(Xnodes[:], mcl.ΦX * mcl.θEs[:,1], title = "Expected value function", xlabel = "k")

       ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Expected value function⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
       ┌────────────────────────────────────────┐ 
   -26 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⡤⠤⠤⠤⠒⠒⠒⠒⠒⠒⠋⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉│ 
       │⠀⠀⠀⠀⠀⢀⣠⠤⠒⠊⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⡠⠖⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⢀⠜⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠌⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⢈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
   -29 │⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       └────────────────────────────────────────┘ 
       ⠀0⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀10⠀ 
       ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀k⠀⠀

In [119]:
# visualize the policy function k'(k,z)
let 

    local fig = up.scatterplot(
        Xnodes[:], Xnext[2][:,1], name = "z = 1.2", color = :blue,
        ylim = (0.0, [Xnext[1] Xnext[2]] |> maximum),
        title = "Policy function k'(k,z)", xlabel = "k", ylabel = "k'(k|z)"
    )

    up.lineplot!(
        fig, Xnodes[:], Xnodes[:], color = :red, name = "45-degree line"
    )

    up.scatterplot!(
        fig, Xnodes[:], Xnext[1][:,1], name = "z = 0.8", color = :green
    )
    
    fig
end

                   ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Policy function k'(k,z)⠀⠀⠀⠀⠀⠀⠀⠀⠀               
                   ┌────────────────────────────────────────┐               
           1.70488 │⠀⠀⠀⠀⠀⠀⡼⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⣀⠤⠤⠤⠒⠒⠒⠒⠉⠉⠉⠉⠉│ z = 1.2       
                   │⠀⠀⠀⠀⠀⢀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⡠⠤⠔⠒⠚⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 45-degree line
                   │⠀⠀⠀⠀⠀⣸⠀⠀⠀⠀⠀⢀⣀⠤⠖⠊⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ z = 0.8       
                   │⠀⠀⠀⠀⠀⡇⠀⠀⣀⠤⠚⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               
                   │⠀⠀⠀⠀⢸⢀⠴⠚⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               
                   │⠀⠀⠀⠀⡟⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               
                   │⠀⠀⢠⢻⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⡤⠤⠤⠤⠤⠤⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒│               
   k'(k|z)         │⠀⢠⠂⡞⠀⠀⠀⠀⣀⡤⠤⠒⠒⠉⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               
                   │⢀⠂⢰⠃⣀⠴⠚⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               
                   │⠠⠀⡼⠊⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               
                   │⢂⢼⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│               